In [1]:
import pandas as pd
import tensorflow as tf
from transformers import BertTokenizer, TFBertModel
import numpy as np
dataset=pd.read_csv('/kaggle/input/en-fr-translation-dataset/en-fr.csv')
dataset.head()

2024-05-21 11:29:20.931690: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-21 11:29:20.931830: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-21 11:29:21.104506: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


,en,fr
0,Changing Lives | Changing Society | How It Wor...,Il a transformé notre vie | Il a transformé la...
1,Site map,Plan du site
2,Feedback,Rétroaction
3,Credits,Crédits
4,Français,English


In [2]:
len(dataset['en'][1])

8

In [3]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = TFBertModel.from_pretrained('bert-base-uncased')
def embedder(text):
    input_ids = tokenizer.encode(text, return_tensors='tf')
    outputs = model(input_ids)
    cls_embedding = outputs.last_hidden_state[:, 0, :]
    #positional encodings
    angle_rates = 1 / np.power(10000, (2 * (768 // 2)) / np.float32(512))
    pos = np.arange(768)[:, np.newaxis]
    angle_rads = pos * angle_rates
    angle_rads[:, 0::2] = np.sin(angle_rads[:, 0::2])
    angle_rads[:, 1::2] = np.cos(angle_rads[:, 1::2])
    pos_encoding = tf.cast(angle_rads[np.newaxis, ...], dtype=tf.float32)
    #adding positional encodings in the original encodings
    cls_embedding += tf.transpose(pos_encoding[:, :768, :][0])
    return cls_embedding
    

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

In [4]:
import math
def encoder(embed):
    # weights of query ,keys and values and other weights
    Wq=tf.random.uniform(shape=(1, 768), minval=-0.5, maxval=0.5)
    Wk=tf.random.uniform(shape=(1, 768), minval=-0.5, maxval=0.5)
    Wv=tf.random.uniform(shape=(1, 768), minval=-0.5, maxval=0.5)
    W0=tf.random.uniform(shape=(1, 768), minval=-0.5, maxval=0.5)
    W1=tf.random.uniform(shape=(1, 768), minval=-0.5, maxval=0.5)
    #Calculating query ,keys and values
    Query=embed*Wq
    Key=embed*Wk
    Value=embed*Wv
    # Scaled-Dot Product Attention
    scores = tf.matmul(Query, Key, transpose_b=True) / math.sqrt(tf.cast(768, tf.float32))
    weights = tf.keras.activations.softmax(scores)
    result=tf.matmul(weights, Value)
    #Add layer
    embed+=result
    #normalize the embeddings
    normalized_embeddings = tf.keras.layers.LayerNormalization()(embed)
    #feed forward layer from scratch
    Layer1=W0*normalized_embeddings
    result1=tf.keras.activations.softmax(Layer1)
    Layer2=W1*normalized_embeddings
    return Layer2

encoder(embedder("Some weights of the PyTorch model were not used when"))

<tf.Tensor: shape=(1, 768), dtype=float32, numpy=
array([[-4.86437082e-02, -5.19792028e-02, -2.09163185e-02,
         3.30187045e-02,  9.87917371e-03,  2.56119436e-03,
         2.70542892e-04,  2.86882240e-02, -8.97494927e-02,
         2.66045332e-02,  7.99951982e-03, -5.64811379e-02,
         5.30055612e-02,  6.82149604e-02, -1.57833502e-01,
        -9.03153326e-04, -8.72723907e-02,  1.99722964e-03,
        -1.35476161e-02,  1.55468332e-03, -1.48588903e-02,
        -1.20698862e-01,  1.95621289e-02, -6.21028021e-02,
        -3.99196386e-01, -1.94003761e-01,  1.07399844e-01,
         8.31760615e-02,  3.90651137e-01, -1.11795381e-01,
        -9.00259987e-02, -2.94604897e-01,  1.73977693e-03,
        -1.03055149e-01,  9.80918668e-03, -3.79031077e-02,
         1.21413469e-01, -1.11184932e-01,  2.11475253e-01,
        -6.24522008e-03, -3.12960185e-02, -3.87227861e-04,
        -1.46489367e-01, -6.26244694e-02,  4.11088131e-02,
        -1.24632418e-02,  1.64808047e+00,  1.78967372e-01,
      

In [5]:
def decoder():
    pass